# Deducing the concentration of PM2.5 based on AOD and weather, etc. with Machine Learning Models

> Note: This notebook is under development.

In [1]:
from osgeo import gdal

In [2]:
import sklearn

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import math

In [5]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [6]:
import csv

## Importing Raster Data
Raster Image I/O

In [7]:
class Grid(object):
    @staticmethod
    def read_img(_file):
        dataset = gdal.Open(_file)
        # 数据描述
        # print(dataset.GetDescription())

        # 图像的列数X与行数Y
        img_width = dataset.RasterXSize
        img_height = dataset.RasterYSize

        # 仿射矩阵
        img_geotrans = dataset.GetGeoTransform()

        # 投影
        img_proj = dataset.GetProjection()

        # 将数据写成数组，对应栅格矩阵
        img_data = dataset.ReadAsArray(0, 0, img_width, img_height)

        # 数据格式大小
        # print(img_data.shape)

        del dataset
        return img_data, img_proj, img_geotrans

    @staticmethod
    def write_img(_file, img_data, img_proj, img_geotrans, _format):
        # 判断栅格数据的数据类型
        if 'int8' in img_data.dtype.name:
            datatype = gdal.GDT_Byte
        elif 'int16' in img_data.dtype.name:
            datatype = gdal.GDT_UInt16
        else:
            datatype = gdal.GDT_Float32

        # 判读数组维数
        if len(img_data.shape) == 3:
            img_bands, img_height, img_width = img_data.shape
        else:
            img_bands, (img_height, img_width) = 1, img_data.shape

        # 创建文件
        # HFA -> .img | GTiff -> .tif
        if _format == 'tif':
            driver = gdal.GetDriverByName("GTiff")
        else:
            driver = gdal.GetDriverByName("HFA")

        dataset = driver.Create(_file, img_width, img_height, img_bands, datatype)

        # 写入仿射变换参数
        dataset.SetGeoTransform(img_geotrans)
        # 写入投影
        dataset.SetProjection(img_proj)
        # 写入数组数据
        # GetRasterBand()
        if img_bands == 1:
            dataset.GetRasterBand(1).WriteArray(img_data)
        else:
            for i in range(img_bands):
                dataset.GetRasterBand(i + 1).WriteArray(img_data[i])

        del dataset

Specifying the extent

In [8]:
x_min = -1225790
x_max = 53952
y_min = -639572
y_max = 685124

In [9]:
img_templete, proj_templete, geotrans_templete = Grid.read_img("./img_data/dem.tif")
line_num, row_num = img_templete.shape

In [10]:
print(line_num, row_num)

2649 2559


In [11]:
img_intercept = img_templete.copy()
img_aod = img_templete.copy()
img_t = img_templete.copy()
img_p = img_templete.copy()
img_ws = img_templete.copy()
img_rh = img_templete.copy()
img_dem = img_templete.copy()
img_ndvi = img_templete.copy()

In [12]:
DEM = img_templete
NDVI, proj, trans = Grid.read_img("./img_data/ndvi.tif")
imageDictTemplate = {
    'Intercept': img_intercept,
    'AOD': img_aod,
    'T': img_t,
    'P': img_p,
    'WS': img_ws,
    'RH': img_rh
}

In [13]:
from sklearn.ensemble import RandomForestClassifier
def RandomForestRegressionDriver(imageDict, )
    global source_data, img_intercept, img_aod, img_t, 
        img_p, img_ws, img_rh, img_dem, img_ndvi, img_local_r,
        line_num, row_num
    

SyntaxError: invalid syntax (<ipython-input-13-495443ae636b>, line 2)

In [14]:
from scipy.stats.stats import pearsonr

In [15]:
help(pearsonr)

Help on function pearsonr in module scipy.stats.stats:

pearsonr(x, y)
    Pearson correlation coefficient and p-value for testing non-correlation.
    
    The Pearson correlation coefficient [1]_ measures the linear relationship
    between two datasets.  The calculation of the p-value relies on the
    assumption that each dataset is normally distributed.  (See Kowalski [3]_
    for a discussion of the effects of non-normality of the input on the
    distribution of the correlation coefficient.)  Like other correlation
    coefficients, this one varies between -1 and +1 with 0 implying no
    correlation. Correlations of -1 or +1 imply an exact linear relationship.
    Positive correlations imply that as x increases, so does y. Negative
    correlations imply that as x increases, y decreases.
    
    The p-value roughly indicates the probability of an uncorrelated system
    producing datasets that have a Pearson correlation at least as extreme
    as the one computed from these da

In [16]:
import csv
help(csv)

Help on module csv:

NAME
    csv - CSV parsing and writing.

MODULE REFERENCE
    https://docs.python.org/3.6/library/csv
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    This module provides classes that assist in the reading and writing
    of Comma Separated Value (CSV) files, and implements the interface
    described by PEP 305.  Although many CSV files are simple to parse,
    the format is not formally defined by a stable specification and
    is subtle enough that parsing lines of a CSV file with something
    like line.split(",") is bound to fail.  The module supports three
    basic APIs: reading, writing, and registration of dialects.
    
    
    DIALECT REGISTRATION:
    
    Readers

In [17]:
def ComputePearsonR():
    fileList = ['14-1', '14-4', '14-7', '15-1', '15-7', '16-1', '16-4', '16-7']
    for file in fileList:
        with open('./table/data-'+ file + '.csv', newline='') as csvfile:
            x_aod = []
            x_at = []
            x_dem = []
            x_rh = []
            x_pr = []
            y = []
            table = csv.DictReader(csvfile, delimiter=',', quotechar='|')
            for row in table:
                x_aod.append(float(row['AOD']))
                x_at.append(float(row['AirTemp']))
                x_dem.append(float(row['DEM']))
                x_rh.append(float(row['RH']))
                x_pr.append(float(row['SeaLevelPr']))
                y.append(float(row['pm2_5']))
            r1 = pearsonr(x_aod, y)
            r2 = pearsonr(x_at, y)
            r3 = pearsonr(x_dem, y)
            r4 = pearsonr(x_rh, y)
            r5 = pearsonr(x_pr, y)
            print(file, r1[1] < 0.05, r2[1] < 0.05, r3[1] < 0.05, r4[1] < 0.05, r5[1] < 0.05)

ComputePearsonR()

14-1 True True True False False
14-4 False True True True True
14-7 True True True True True
15-1 False True True True False
15-7 True True True True True
16-1 True True True False True
16-4 True True True True False
16-7 True True True True True


In [18]:
from scipy import stats

In [19]:
def readImageDataAsArray(filePath):
    data = Grid.read_img(filePath)[0]
    shape = np.shape(data)
    array = data.ravel()
    print("Original Shape: \t" + str(shape))
    print("Flattened array size: \t" + str(np.shape(array)[0]))
    print("Array:\t" + str(array))
    return array, shape

In [20]:
ndviArray, shape = readImageDataAsArray("./img_data/ndvi.tif")
demArray = readImageDataAsArray("./img_data/dem.tif")[0]

Original Shape: 	(2649, 2559)
Flattened array size: 	6778791
Array:	[ 2158  1953  1982 ...  7987  6740 -3000]
Original Shape: 	(2649, 2559)
Flattened array size: 	6778791
Array:	[1458 1454 1458 ...  123   97  104]


In [21]:
print(ndviArray)
print(ndviArray[6778790])

[ 2158  1953  1982 ...  7987  6740 -3000]
-3000


In [22]:
def importAllData(date, hour):
    imageDict = {
        "aod": readImageDataAsArray("./img_data/aod-" + str(date) + "-" + str(hour) + ".tif"),
        "t": readImageDataAsArray("./img_data/t-" + str(date) + "-" + str(hour) + ".tif"),
        "rh": readImageDataAsArray("./img_data/rh-" + str(date) + "-" + str(hour) + ".tif"),
        "p": readImageDataAsArray("./img_data/p-" + str(date) + "-" + str(hour) + ".tif"),
        "ws": readImageDataAsArray("./img_data/ws-" + str(date) + "-" + str(hour) + ".tif")
    }
    return imageDict

In [23]:
def dict2Mat(imageDict):
    matrix = np.mat([ndviArray, demArray, imageDict["aod"], imageDict["t"], imageDict["rh"], imageDict["p"], imageDict["ws"]])
    return matrix

In [44]:
def loadSample(date, hour):       
    with open('./table/data-'+ str(date) + "-" + str(hour) + '.csv', newline='') as csvfile:
        x_aod = []
        x_at = []
        x_dem = []
        x_rh = []
        x_pr = []
        x_ndvi = []
        x_ws = []
        y = []
        ones = []
        table = csv.DictReader(csvfile, delimiter=',', quotechar='|')
        for row in table:
            ones.append(1.0)
            x_aod.append(float(row['AOD']))
            x_at.append(float(row['AirTemp']))
            x_dem.append(float(row['DEM']))
            x_rh.append(float(row['RH']))
            x_pr.append(float(row['SeaLevelPr']))
            x_ndvi.append(float(row['NDVI']))
            x_ws.append(float(row['WindSpeed']))
            y.append(float(row['pm2_5']))
        xt = [np.array(ones),
              x_ndvi, 
              x_dem, 
              x_aod, 
              x_at, 
              x_rh, 
              x_pr, 
              x_ws]
        xt_np = np.mat(xt)
        x = np.transpose(xt_np)
        y_np = np.transpose(np.mat(y))
        print(np.shape(x))
        print(x)
#         print(xt_np)
#         print(y_np)
        return x, xt_np, y_np

In [45]:
x141, xt141, y141 = loadSample(14,1)

(295, 8)
[[1.00000000e+00 1.58100000e+03 2.90000000e+01 ... 4.91983986e+01
  1.02060000e+04 3.00086002e+01]
 [1.00000000e+00 2.11700000e+03 1.20000000e+01 ... 5.00164986e+01
  1.02122002e+04 3.34331017e+01]
 [1.00000000e+00 2.51200000e+03 2.40000000e+01 ... 5.87461014e+01
  1.02230000e+04 2.14207001e+01]
 ...
 [1.00000000e+00 2.46200000e+03 9.60000000e+01 ... 4.51273994e+01
  1.02184004e+04 2.02595997e+01]
 [1.00000000e+00 1.50200000e+03 3.90000000e+01 ... 4.70787010e+01
  1.02010000e+04 1.50045996e+01]
 [1.00000000e+00 1.54100000e+03 1.10000000e+01 ... 4.65489006e+01
  1.02120000e+04 2.50328999e+01]]


In [70]:
class MLR:     
    def __init__(self, x, y = None):
        
        self.init()
        if (y is None):
            self.x = np.array(matrix[:,0:-1])
            self.y = np.array(matrix[:,-1])
            shape = np.shape(matrix)
            self.k = shape[1] - 2
            self.n = shape[0]
        else:
            self.x = x
            self.y = y
            self.k = np.shape(x)[1] - 1
            self.n = np.shape(x)[0]
    
    def init(self):
        self.r2 = 0.
        self.t = []
        self.p_t = []
        self.f = 0.
        self.p_f = 0.
        self.ess = 0.
        self.rss = 0.
        self.tss = 0.
        self.r2_adj = 0.

        
    def OLS(self):
        x = self.x
        y = self.y
        x_t = np.transpose(x)
        xt_x_inv = np.matmul(x_t, x)
        xt_x_inv = np.linalg.inv(xt_x_inv)
        beta = np.matmul(xt_x_inv, x_t)
        beta = np.matmul(beta, y)
        self.beta = beta
        y_e = self.PredictionE(x)
        deviation1 = self.y - y_e
        print(deviation1)
        deviation = deviation1 ** 2
        self.rss = np.sum(deviation)
        mean = np.mean(y)
        self.tss = np.sum((y-mean) * (y-mean))
        self.ess = self.tss - self.rss
        self.f = (self.ess / self.k) / (self.rss / (self.n - self.k - 1))
        self.r2 = 1 - self.rss / self.tss
        self.r2_adj = 1 - (self.rss / (self.n - self.k - 1)) / (self.tss / (self.n - 1))
        self.p_f = 1 - stats.f.cdf(self.f, self.k, (self.n - self.k - 1))
        sigma2 = self.rss / (self.n - self.k - 1)
        beta_array = np.array(np.transpose(beta)[0])
        cii = []
        for i in range(0, self.k + 1):
            cii.append(xt_x_inv[i][i])
        cii_np = np.array(cii)
        self.t = beta_array / np.sqrt(cii_np * sigma2)
        self.p_t = 1 - stats.t.cdf(np.absolute(self.t), self.n - self.k - 1)
        
    def PredictionE(self, x):
        return np.matmul(x, self.beta)

In [75]:
mlr141 = MLR(np.array(x141), np.array(y141))

In [76]:
np.shape(x141)

(295, 8)

In [77]:
np.shape(y141)

(295, 1)

In [78]:
y141

matrix([[47.8],
        [32.5],
        [15.7],
        [35. ],
        [39.4],
        [34.2],
        [36.9],
        [35.3],
        [39.1],
        [48.2],
        [19. ],
        [24.6],
        [42.2],
        [40.5],
        [46.7],
        [37.5],
        [33.2],
        [22.3],
        [16.9],
        [ 4.8],
        [10. ],
        [ 9.8],
        [ 9.9],
        [ 8.2],
        [ 5.4],
        [ 7.6],
        [ 8.3],
        [ 9. ],
        [11.7],
        [ 7.3],
        [13.5],
        [ 9.9],
        [14.2],
        [12.5],
        [10.8],
        [22.4],
        [11.5],
        [14. ],
        [16.3],
        [14. ],
        [50.3],
        [35.5],
        [58.5],
        [60.8],
        [52.8],
        [61.6],
        [38.1],
        [47.2],
        [46.4],
        [48.4],
        [34.1],
        [58. ],
        [52.7],
        [26.5],
        [31.7],
        [64.6],
        [55.9],
        [42.5],
        [45.2],
        [27.7],
        [29.2],
        [35.5],
        

In [79]:
mlr141.OLS()

[[ 3.56881905e+00]
 [-7.45918273e+00]
 [-2.09916114e+01]
 [-3.04968771e+00]
 [-6.09169278e-01]
 [-6.16187752e+00]
 [-6.02087394e+00]
 [-5.11660893e+00]
 [-3.21510208e+00]
 [ 4.98401469e+00]
 [-1.13306922e+01]
 [-1.39782173e+01]
 [ 3.05157088e+00]
 [ 7.61926268e-01]
 [ 6.28664253e+00]
 [-8.39711427e-01]
 [-5.52724966e+00]
 [-1.46942059e+01]
 [-2.30771540e+01]
 [-1.60805193e+01]
 [-1.14192266e+01]
 [-6.14393251e+00]
 [-8.84671187e+00]
 [-1.11727761e+01]
 [-1.85872632e+01]
 [-1.68319172e+01]
 [-1.90358183e+01]
 [-1.78835259e+01]
 [-1.36996415e+01]
 [ 8.30125352e-01]
 [-1.38382662e+01]
 [-1.31290957e+01]
 [-1.28164518e+01]
 [-6.75103474e+00]
 [-1.50833952e+01]
 [ 4.93762762e+00]
 [-4.14478000e+00]
 [-1.32128664e+01]
 [-1.13497731e+01]
 [-1.33133768e+01]
 [ 8.59875102e+00]
 [ 6.70141834e+00]
 [ 1.94061359e+01]
 [ 2.01470033e+01]
 [ 1.37088202e+01]
 [ 1.71490901e+01]
 [ 1.34467114e+00]
 [ 6.70046894e+00]
 [ 7.05893541e+00]
 [ 9.33471202e+00]
 [-9.03646730e+00]
 [ 1.75472128e+01]
 [ 5.5374301

In [80]:
mlr141.r2

0.5645581412776206